In [20]:
import xml.etree.ElementTree as ET
from functools import reduce

In [21]:
url = '/home/carlosdev/Descargas/Stackoverflow/Stack Overflow 11-2010/112010 Meta Stack Overflow/posts.xml'
tree = ET.parse(url)
root = tree.getroot()

In [22]:
#mapper
#extrayendo info del xml
post_view = []
def mapper(id):
    '''
    Función que retorna lista mapeada

    Args:
        id (list): lista de id_post y cantidad de vistas

    Returns:
        list: lista de id_post y cantidad de vistas
    '''
    return id
#itero sobre el elemento que contiene todos los post del .xml
for i in root:
    var = map(mapper, [i.attrib['Id'], i.attrib['ViewCount']])
    post_view.append(list(var))

In [23]:
def reducex(a, b):
    '''
    Función que devuelve el más grande de dos numeros

    Args:
        a (list): lista de 2 elementos(id del post, cantidad de vistas)
        b (list): lista de 2 elementos(id del post, cantidad de vistas)

    Returns:
        list: lista con cantidad de vistas mayor
    '''
    if int(a[1]) > int(b[1]):
        return a
    return b
#ordeno la lista 
post_view2 = sorted(post_view, key=lambda x: int(x[1]), reverse=True)
#aplico el reduce y muestro los primeros 10
for i in range(10):
    print(reduce(reducex, post_view2))
    post_view2.pop(0)

['28625', '33344']
['37328', '28372']
['31913', '26601']
['9134', '20536']
['1777', '19695']
['2267', '15180']
['7931', '12584']
['61142', '9918']
['20420', '8903']
['53346', '8619']
